In [ ]:
# RAND Score Clustering Analysis
#Imports
import sys
print(sys.executable)
import pandas as pd
import numpy as np
import umap
import hdbscan
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score, rand_score
import plotly.express as px
print("all good")

c:\Users\ellio\AppData\Local\Programs\Python\Python312\python.exe


C:\Users\ellio\AppData\Local\Temp\ipykernel_22592\113384497.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\ellio\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


all good


In [2]:
# Load and scale data
import re
import ast

df = pd.read_csv("../data/features_updated.csv")
file_names = df["file_name"]
X = df.drop(columns=["file_name"])
def safe_parse_dict(s):
    """Parse dict-like strings safely. Returns {} if parsing fails."""
    if pd.isna(s):
        return {}
    s = str(s).strip()
    if not s.startswith("{"):
        return {}
    
    # Remove np.float64(...) wrappers -> just the number inside
    s = re.sub(r"np\.float64\(([^)]+)\)", r"\1", s)
    
    # Replace 'nan' with None (literal_eval can't parse bare nan)
    s = s.replace("nan", "None")
    
    try:
        out = ast.literal_eval(s)
        return out if isinstance(out, dict) else {}
    except Exception:
        return {}

# Find dict-like columns
dict_cols = [c for c in X.columns if X[c].astype(str).str.strip().str.startswith("{").any()]
print("Expanding:", dict_cols)

for c in dict_cols:
    parsed = X[c].apply(safe_parse_dict)
    expanded = pd.json_normalize(parsed).add_prefix(f"{c}_")
    X = X.drop(columns=[c]).join(expanded)

# Convert everything numeric
X = X.apply(pd.to_numeric, errors="coerce").fillna(0)

print("Final feature count:", X.shape[1])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Scaling complete:", X_scaled.shape)

Expanding: ['a0_band_powers', 'a0_band_ratios', 'a0_spectrogram_features', 'a0_wavelet_energy_features', 'a1_band_powers', 'a1_band_ratios', 'a1_spectrogram_features', 'a1_wavelet_energy_features']
Final feature count: 73
Scaling complete: (446, 73)


In [3]:
#UMAP
umap_model = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    n_components=2,
    metric="euclidean",
    random_state=42
)
X_umap = umap_model.fit_transform(X_scaled)

c:\Users\ellio\AppData\Local\Programs\Python\Python312\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [4]:
#HDBSCAN tuned for 2 clusters
# need to tune min_cluster_size and min_samples until we get exactly 2 clusters
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=50, # increase to merge smaller clusters into 1
    min_samples=10,
    metric="euclidean"
)
predicted_labels = clusterer.fit_predict(X_umap)
n_clusters = len(set(predicted_labels) - {-1})
n_noise = (predicted_labels == -1).sum()
print(f"Number of clusters: {n_clusters}")
print(f"Noise points: {n_noise} / {len(predicted_labels)}")

Number of clusters: 2
Noise points: 0 / 446


In [ ]:
#Visualization of clusters
plot_df = pd.DataFrame({
    "UMAP-1": X_umap[:, 0],
    "UMAP-2": X_umap[:, 1],
    "cluster": predicted_labels.astype(str),
    "file_name": file_names.values
})
fig = px.scatter(plot_df, x="UMAP-1", y="UMAP-2", color="cluster",
                 hover_data=["file_name"], title="UMAP + HDBSCAN (2-cluster target)")
fig.update_layout(width=800, height=600)

html_template = """
<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <div id="plot"></div>
    <div id="image-container" style="margin-top: 20px; text-align: center;">
        <p id="image-label" style="font-family: Arial; color: #555;"></p>
        <img id="clicked-image" src="" style="display:none; max-width: 900px; border: 1px solid #ccc; border-radius: 6px;"/>
        <p id="no-image-msg" style="font-family: Arial; color: #999; display:none;">No plot found for this file.</p>
    </div>
    <script>
        var plotData = PLOTLY_JSON;
        Plotly.newPlot('plot', plotData.data, plotData.layout);

        document.getElementById('plot').on('plotly_click', function(data) {
            var point = data.points[0];
            var fileName = point.customdata[0];
            var baseName = fileName.replace(/\\.csv$/, '');
            var imgPath = '../visuals/boiling_plots/' + baseName + '.png';

            var img = document.getElementById('clicked-image');
            var label = document.getElementById('image-label');
            var noMsg = document.getElementById('no-image-msg');

            img.src = imgPath;
            img.style.display = 'block';
            noMsg.style.display = 'none';
            label.textContent = 'File: ' + fileName;

            img.onerror = function() {
                img.style.display = 'none';
                noMsg.style.display = 'block';
                label.textContent = 'File: ' + fileName;
            };
        });
    </script>
</body>
</html>
"""

import plotly.io as pio
import json
from pathlib import Path

fig_json = json.loads(pio.to_json(fig))
html_content = html_template.replace("PLOTLY_JSON", json.dumps(fig_json))

output_path = Path("../visuals/cluster_plot.html")
output_path.parent.mkdir(parents=True, exist_ok=True)
with open(output_path, "w") as f:
    f.write(html_content)

print(f"Interactive plot saved to {output_path} — open in a browser to use click functionality.")

Interactive plot saved to ..\visuals\cluster_plot.html — open in a browser to use click functionality.


c:\Users\ellio\AppData\Local\Programs\Python\Python312\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [25]:
# Load ground truth from Excel and map to binary

# Expects: column A = file name, column B = category number (1-8)
gt_df = pd.read_excel("../data/Labeling.xlsx")  

# Drop unlabeled rows
gt_df = gt_df.dropna(subset=["Label"])
gt_df["Label"] = gt_df["Label"].astype(int)

# Add "MATLAB " prefix to match features.csv file_name format
gt_df["file_name"] = "MATLAB " + gt_df["File"]

# Category mapping:
# 1 = Single Rhythmic        → rhythmic
# 2 = Double Rhythmic         → rhythmic
# 3 = Random                  → non-rhythmic
# 4 = Rhythmic with Climax    → rhythmic
# 5 = Noise                   → non-rhythmic
# 6 = 1 Rhythmic with Random  → rhythmic
# 7 = Triple Rhythmic         → rhythmic
# 8 = Transition              → EXCLUDE (ambiguous)

category_names = {
    1: "Single Rhythmic",
    2: "Double Rhythmic",
    3: "Random",
    4: "Rhythmic with Climax",
    5: "Noise",
    6: "1 Rhythmic with Random",
    7: "Triple Rhythmic",
    8: "Transition"
}

rhythmic_ids = [1, 2, 4, 6, 7]
non_rhythmic_ids = [3, 5]

gt_df["ground_truth_binary"] = gt_df["Label"].apply(
    lambda x: 1 if x in rhythmic_ids else (0 if x in non_rhythmic_ids else np.nan)
)

# Drop Transition (8) samples
gt_df = gt_df.dropna(subset=["ground_truth_binary"])
gt_df["ground_truth_binary"] = gt_df["ground_truth_binary"].astype(int)

print(f"Labeled samples: {len(gt_df)}")
print(f"  Rhythmic (1):     {(gt_df['ground_truth_binary'] == 1).sum()}")
print(f"  Non-rhythmic (0): {(gt_df['ground_truth_binary'] == 0).sum()}")

Labeled samples: 283
  Rhythmic (1):     177
  Non-rhythmic (0): 106


In [26]:
#RAND calculation
pred_df = pd.DataFrame({
    "file_name": file_names,
    "predicted_cluster": predicted_labels
})

merged = pd.merge(pred_df, gt_df[["file_name", "ground_truth_binary"]], on="file_name", how="inner")

# Optional: also drop HDBSCAN noise points (-1)
# merged = merged[merged["predicted_cluster"] != -1]

print(f"\nSamples matched: {len(merged)} / {len(file_names)}")
print(f"Unmatched (no ground truth or Transition): {len(file_names) - len(merged)}")

ri = rand_score(merged["ground_truth_binary"], merged["predicted_cluster"])
ari = adjusted_rand_score(merged["ground_truth_binary"], merged["predicted_cluster"])

print(f"\nRAND Index:          {ri:.4f}")
print(f"Adjusted RAND Index: {ari:.4f}")


Samples matched: 283 / 446
Unmatched (no ground truth or Transition): 163

RAND Index:          0.7316
Adjusted RAND Index: 0.4566


The Rand Index (0.73) means that about 73% of all possible pairs of samples were grouped consistently between the unsupervised clustering results and the labeled boiling regimes.
**The Rand Index doesn't account for agreement that could occur simply by chance
The ARI (0.4566) means that a moderate level of agreement between the unsupervised HDBSCAN clusters and the labeled regimes
    -An ARI near 0 would indicate random clustering
    -An ARI near 1 would indicate near-perfect agreement.